# Initialize JlBox environment

In [ ]:
using Pkg
Pkg.activate("..")
Pkg.build("JlBox")

# Gas phase simulation
- Simulation config:
    - Simulation period: 10800 seconds from midday 12:00
    - Simulation condition: temperature = 298.15K, solar declination angle = 23.79deg, latitude = 50deg
    - Initial condition: 18 ppm Ozone, 30 ppm Alpha-Pinene
- Solver config:
    - Solver: TRBDF2
    - Tolerance: relative tol = 1e-6, absolute tol = 1e-4
    - Time step: initial timestep = 1e-6, max timestep = 100.0
    - Jacobian: dense, analytical
- Output:
    - df: concentration of gases (unit: molecules/cc)

In [ ]:
using JlBox, OrdinaryDiffEq
function configure_gas()
    file=joinpath(@__DIR__,"../data/MCM_APINENE_O3.eqn.txt")#"MCM_test.eqn.txt"MCM_APINENE.eqn.txt"MCM_mixed_test.eqn.txt"MCM_ISOPRENE.eqn.txt
    temp=298.15 # Kelvin
    RH=0.5 # RH/100% [0 - 0.99]
    hour_of_day=12.0 # Define a start time  24 hr format
    start_time=hour_of_day*60*60 # seconds, used as t0 in solver
    simulation_time= 10800.0 # seconds
    batch_step=100.0 # seconds
    temp_celsius=temp-273.15
    Psat=610.78*exp((temp_celsius/(temp_celsius+238.3))*17.2694)# Saturation VP of water vapour, to get concentration of H20
    Pw=RH*Psat
    Wconc=0.002166*(Pw/(temp_celsius+273.16))*1.0e-6 #kg/cm3
    H2O=Wconc*(1.0/(18.0e-3))*6.0221409e+23#Convert from kg to molecules/cc
    Cfactor=2.55e+10 #ppb-to-molecules/cc
    reactants_initial_dict=Dict(["O3"=>18.0,"APINENE"=>30.0])#ppm ["O3"=>18.0,"APINENE"=>30.0])BUT1ENE"C5H8
    constant_dict=Dict([(:temp,temp),(:H2O,H2O)])
    dec=23.79
    lat=50.0
    photolysis_config=JlBox.DiurnalPhotolysisConfig(dec, lat)
    solver=TRBDF2(autodiff=false)
    #solver=Sundials.CVODE_BDF()
    reltol=1e-6
    abstol=1.0e-4
    dtinit=1e-6
    dtmax=100.0
    positiveness=false
    use_jacobian=true
    sparse=false
    config=JlBox.GasConfig(file,temp,RH,start_time,simulation_time,batch_step,
                       H2O,Cfactor,reactants_initial_dict,constant_dict,photolysis_config)
    solverconfig=JlBox.SolverConfig(solver,sparse,reltol,abstol,dtinit,dtmax,positiveness)
    config,solverconfig
end

config, solverconfig = configure_gas()
sol_gas, reactants2ind_gas, _ = JlBox.run_simulation(config, solverconfig);

In [ ]:
df = JlBox.postprocess_gas(sol_gas, reactants2ind_gas, config)
df[:,[:APINENE,:O3,:OH,:PINONIC,:O,:O1D,:NO2]]

## Plotting

Install `Plots` package

In [ ]:
Pkg.add("Plots")

### Plot time series for Alpha-Pinene & PINONIC

Same result can be found in : https://github.com/loftytopping/PyBox/blob/master/images/Example_deafult_gas_simulation.png

In [ ]:
using Plots
Cfactor=2.55e+10 #ppb-to-molecules/cc
t_index = range(0, stop = 10800/3600, length = 109)
plot(t_index, log10.(df[!,:APINENE]), label = "Alpha-Pinene")
plot!(t_index, log10.(df[!,:PINONIC]), label = "PINONIC")
plot!(t_index, log10.(df[!,:O3]), label = "O3")
plot!(t_index, log10.(df[!,:NO2]), label = "NO2")
plot!(t_index, log10.(df[!,:NO]), label = "NO")
xlabel!("Time (hour)")
ylabel!("Log10(Concentration in molecules/cc)")

# Gas+Aerosol Simulation
- Simulation config:
    - Simulation period: 3600 seconds from midday 12:00
    - Simulation condition: temperature = 288.15K, solar declination angle = 23.79deg, latitude = 50deg
    - Initial condition: 18 ppm Ozone, 30 ppm Alpha-Pinene, 50% relative humidity
    - \# Size bins: 16
    - Nuclei profile: lognormal size distribution of 100 particles/cc, mean size = 0.2 um, size std = 2.2 um, lower size = 0.01 um, upper size = 1.0 um
    - Nuclei material: ammonium sulphate
- Solver config:
    - Solver: CVODE_BDF
    - Tolerance: relative tol = 1e-4, absolute tol = 1e-2
    - Time step: initial timestep = 1e-6, max timestep = 100.0
    - Jacobian: dense, analytical
- Output:
    - df_gas: concentration of gases (unit: molecules/cc)
    - df_SOA: Secondary Organic Aerosol mass (unit: ug/m3), and mass distribution for each size bin (unit: ug/m3)
    - df_size: Size bins (unit: m)

In [ ]:
using JlBox
using Sundials

function configure_aerosol()
    file=joinpath(@__DIR__,"../data/MCM_APINENE.eqn.txt")#"MCM_test.eqn.txt"MCM_BCARY.eqn.txt
    temp=288.15 # Kelvin
    RH=0.5 # RH/100% [0 - 0.99]
    hour_of_day=12.0 # Define a start time  24 hr format
    start_time=hour_of_day*60*60 # seconds, used as t0 in solver
    simulation_time= 3600.0 # seconds
    batch_step=300.0 # seconds
    temp_celsius=temp-273.15
    Psat_w=610.78*exp((temp_celsius/(temp_celsius+238.3))*17.2694)# Saturation VP of water vapour, to get concentration of H20
    Pw=RH*Psat_w
    Wconc=0.002166*(Pw/(temp_celsius+273.16))*1.0e-6 #kg/cm3
    H2O=Wconc*(1.0/(18.0e-3))*6.0221409e+23#Convert from kg to molecules/cc
    Cfactor= 2.55e+10 #ppb-to-molecules/cc
    reactants_initial_dict=Dict(["O3"=>18.0,"APINENE"=>30.0,"H2O"=>H2O/Cfactor])#ppb BUT1ENE APINENE
    constant_dict=Dict([(:temp,temp)])
    dec=23.79
    lat=50.0
    photolysis_config=JlBox.DiurnalPhotolysisConfig(dec, lat)
    num_bins=16

    #Lognormal Distribution
    total_conc=100 #Total particles per cc
    size_std=2.2 #Standard Deviation
    lowersize=0.01 #microns
    uppersize=1.0 #microns
    meansize=0.2 #microns

    # - Specify the core material. 
    # This code is currently setup to consider *ammonium sulphate* as the core
    y_core_init=1.0e-3.+zeros(Float64,num_bins) #Will hold concentration of core material, only initialise here [molecules/cc] 
    core_density_array=1770.0.+zeros(Float64,num_bins) #[kg/m3] - need to make sure this matches core definition above
    core_mw=132.14.+zeros(Float64,num_bins) #[g/mol]
    core_dissociation=3.0 #Define this according to choice of core type. Please note this value might change

    vp_cutoff=-6.0
    sigma=72.0e-3 # Assume surface tension of water (mN/m) ???
    property_methods=Dict("bp"=>"joback_and_reid","vp"=>"nannoolal","critical"=>"nannoolal","density"=>"girolami")
    config=JlBox.AerosolConfig(file,temp,RH,start_time,simulation_time,batch_step,
                           H2O,Cfactor,reactants_initial_dict,constant_dict,photolysis_config,num_bins,
                           total_conc,size_std,lowersize,uppersize,meansize,y_core_init,
                           core_density_array,core_mw,core_dissociation,vp_cutoff,
                           sigma,property_methods)
    config
end

function configure_aerosol_solver_dense()
    solver=Sundials.CVODE_BDF()#OrdinaryDiffEq.TRBDF2(autodiff=false)
    sparse=false
    reltol=1e-4
    abstol=1.0e-2
    dtinit=1e-6
    dtmax=100.0
    positiveness=false
    diff_method="fine_analytical"
    solverconfig=JlBox.SolverConfig(solver,sparse,reltol,abstol,dtinit,dtmax,positiveness,diff_method)
    solverconfig
end

config = configure_aerosol()
solverconfig = configure_aerosol_solver_dense()
@time sol, reactants2ind, param_dict = JlBox.run_simulation(config, solverconfig);

In [ ]:
df_gas = JlBox.postprocess_gas(sol, reactants2ind, config)
df_SOA = JlBox.postprocess_aerosol(sol, param_dict, config)
df_size = JlBox.postprocess_aerosol_size_dist(sol, param_dict, config)
df_SOA

In [ ]:
using Plots
plot(df_SOA[!,:Time]/60, df_SOA[!,:SOA], label = "")
xlabel!("Time (minute)")
ylabel!("SOA mass (ug/m3)")

In [ ]:
using Plots
plot(df_size[!,:Time]/60, log10.(Matrix(df_size[:,2:end])), label = "")
xlabel!("Time (minute)")
ylabel!("log10(Bin size in m)")